In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("chanchito-feliz").getOrCreate()
spark

24/10/19 17:06:16 WARN Utils: Your hostname, MacBook-Pro-de-Nahuel.local resolves to a loopback address: 127.0.0.1; using 192.168.18.36 instead (on interface en7)
24/10/19 17:06:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/19 17:06:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [4]:
#leer el archivo

app = spark.read.csv(path        = "/Users/franca/Documents/ETL-con-pyspark/googleplaystore.csv",
                         inferSchema = True, header = True)
app.show(5, truncate = True)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|  August 1, 2018|             1.2.4|4.0.3 and up|
|Ske

In [5]:
app.limit(4).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up


In [6]:
#explorar el df
app.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [7]:
#numero de filas
app.count()


10841

In [8]:
#numero de columnas
len(app.columns)

13

In [9]:
#contar valores nulos de cada columna

nulos = app.select([sum(col(c).isNull().cast("int")).alias(c) for c in app.columns])
nulos.show()

+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|App|Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|  0|       0|     0|      0|   0|       0|   0|    0|             1|     0|           0|          1|          1|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+




app.columns: Esta expresión accede a la lista de todas las columnas del DataFrame.

for c in app.columns: Recorremos cada columna del DataFrame con el nombre c, en un loop para aplicar la lógica a cada una.

col(c): Usamos la función col() para referirnos a cada columna por su nombre c.
isNull(): Esta función verifica si el valor de cada fila en la columna es nulo. Devuelve True si es nulo, y False si no lo es.

cast("int"): Dado que True y False no son numéricos, necesitamos convertirlos en enteros. Al hacer el cast("int"), se convierte True en 1 y False en 0. Así, podemos contar los nulos al sumar los valores 1.

sum(): Se utiliza para sumar los valores de 1 generados por los nulos en cada columna.

alias(c): Le damos un alias al resultado de la suma, que será el nombre de la columna original c. Esto sirve para que el resultado final tenga el nombre de cada columna.

In [24]:
#cantidad de duplicados por cada columna 

#contar numero total de filas 

total_filas = app.count()

#lsta para almacenar las columnas que tienen duplicaods 

columnas_con_duplicados = []

#verificar duplicados en cada columna 

for c in app.columns:
    #contar valores unicos en cada columna
    unicos = app.select(c).distinct().count()
    
    #si el numero de valores unicos es menor que el total de filas, hay duplicados en esa columna
    if unicos < total_filas:
        columnas_con_duplicados.append(c)
        
#mostrar columnas duplicadas

print(f"Las columnas con duplicados son: {', '.join(columnas_con_duplicados)}")


Las columnas con duplicados son: App, Category, Rating, Reviews, Size, Installs, Type, Price, Content, Genres, Last Updated


In [13]:
# Contar valores únicos por columna
for columna in app.columns:
    unicos = app.select(columna).distinct().count()
    print(f"Valores únicos en {columna}: {unicos}")

Valores únicos en App: 9660
Valores únicos en Category: 36
Valores únicos en Rating: 43
Valores únicos en Reviews: 6002
Valores únicos en Size: 464
Valores únicos en Installs: 24
Valores únicos en Type: 6
Valores únicos en Price: 95
Valores únicos en Content Rating: 9
Valores únicos en Genres: 121
Valores únicos en Last Updated: 1379
Valores únicos en Current Ver: 2836
Valores únicos en Android Ver: 37


In [14]:
#Eliminar columnas que se que no voy a usar

eliminar_columnas = app.drop('Current Ver', 'Android Ver')

In [17]:
eliminar_columnas.limit(4).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018"
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018"
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018"
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018"


In [18]:
#corregir types de las columnas

data_schema = list((StructField("App"     ,      StringType(), True),
                    StructField("Category",      StringType(), True),
                    StructField("Rating"  ,      IntegerType(), True),
                    StructField("Reviews" ,      IntegerType(), True),
                    StructField("Size" ,         IntegerType(), True),
                    StructField("Installs",      IntegerType(), True),
                    StructField("Type"    ,      StringType(), True),
                    StructField("Price"   ,      IntegerType(), True),
                    StructField("Content" ,      StringType(), True),
                    StructField("Genres"  ,      StringType(), True),
                    StructField("Last Updated" , DateType(), True)))

final_struc = StructType(fields = data_schema)


In [23]:
app = spark.read.csv(path        = "/Users/franca/Documents/ETL-con-pyspark/googleplaystore.csv",
                                 schema = final_struc)

app.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: date (nullable = true)



In [30]:
# Contar el número total de filas antes de eliminar duplicados
total_filas_antes = app.count()

#eliminar duplicados

app_sin_duplicados = app.dropDuplicates()

# Contar el número de filas después de eliminar duplicados
total_filas_despues = app_sin_duplicados.count()

# Mostrar la diferencia
if total_filas_antes > total_filas_despues:
    print(f"Duplicados eliminados. Filas antes: {total_filas_antes}, Filas después: {total_filas_despues}")
else:
    print("No se encontraron duplicados para eliminar.")

No se encontraron duplicados para eliminar.
